In [ ]:
# Content-based car recommender op basis van merk, model, brandstof, vermogen, budget

## Stap-voor-stap implementatie van een content-based recommender

In [ ]:
!pip install scikit-learn



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Stap 1: Importeer benodigde libraries


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Stap 2: Laad de CSV data en bekijk de structuur
df_export = pd.read_csv('Cleaned_Car_Data_For_App.csv')

# Hernoem 'prijs' naar 'budget' voor consistentie met de recommender
if 'prijs' in df_export.columns:
    df_export = df_export.rename(columns={'prijs': 'budget'})

# Controleer de structuur
print("Dataframe info:")
print(df_export.info())
print("\nEerste rijen:")
print(df_export.head())
print("\nKolommen:", df_export.columns.tolist())
print(f"\nAantal rijen: {len(df_export)}")


Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20755 entries, 0 to 20754
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   merk       20755 non-null  object 
 1   model      20755 non-null  object 
 2   bouwjaar   20755 non-null  int64  
 3   type_auto  20755 non-null  object 
 4   brandstof  20755 non-null  object 
 5   vermogen   20415 non-null  float64
 6   budget     20755 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB
None

Eerste rijen:
        merk    model  bouwjaar  type_auto brandstof  vermogen   budget
0      lexus   rx 450      2010       jeep    hybrid       0.0  13328.0
1  chevrolet  equinox      2011       jeep    petrol       0.0  16621.0
2      honda      fit      2006  hatchback    petrol       0.0   8467.0
3       ford   escape      2011       jeep    hybrid       0.0   3607.0
4      honda      fit      2014  hatchback    petrol       0.0  11726.0

K

In [ ]:
# Stap 3: ANALYSE VERMOGEN - Onderzoek betrouwbaarheid van vermogen data
# Voordat we blind imputeren, analyseren we eerst hoeveel auto's betrouwbaar vermogen hebben
# Dit helpt beslissen of we auto's zonder vermogen kunnen droppen zonder de dataset te klein te maken

print("=" * 80)
print("ANALYSE VERMOGEN - Betrouwbaarheid van vermogen data")
print("=" * 80)

# Bereken statistieken over vermogen kolom
totaal_rijen = len(df_export)
rijen_met_nan = df_export['vermogen'].isna().sum()
rijen_met_nul = (df_export['vermogen'] == 0).sum()
rijen_met_betrouwbaar_vermogen = ((df_export['vermogen'].notna()) & (df_export['vermogen'] > 0)).sum()

# Bereken percentages
percentage_nan_of_nul = ((rijen_met_nan + rijen_met_nul) / totaal_rijen) * 100
percentage_betrouwbaar = (rijen_met_betrouwbaar_vermogen / totaal_rijen) * 100

# Print alle statistieken
print(f"\nTotaal aantal rijen in df_export: {totaal_rijen}")
print(f"Aantal rijen met NaN in vermogen: {rijen_met_nan}")
print(f"Aantal rijen met vermogen == 0: {rijen_met_nul}")
print(f"Percentage met NaN of 0 als vermogen: {percentage_nan_of_nul:.2f}%")
print(f"\nAantal rijen met betrouwbaar vermogen (vermogen > 0): {rijen_met_betrouwbaar_vermogen}")
print(f"Percentage met betrouwbaar vermogen: {percentage_betrouwbaar:.2f}%")

# Bepaal of we kunnen filteren op betrouwbaar vermogen
# Criteria: meer dan 5.000 rijen OF meer dan 80% van de data
min_absoluut_aantal = 5000
min_percentage = 80.0

kan_filteren = (rijen_met_betrouwbaar_vermogen > min_absoluut_aantal) or (percentage_betrouwbaar > min_percentage)

print("\n" + "-" * 80)
if kan_filteren:
    print(f"✅ BESLISSING: Er zijn voldoende auto's met betrouwbaar vermogen")
    print(f"   ({rijen_met_betrouwbaar_vermogen} rijen = {percentage_betrouwbaar:.2f}%)")
    print(f"   We maken df_power_clean met alleen auto's waar vermogen > 0")
    
    # Maak nieuwe DataFrame met alleen auto's met betrouwbaar vermogen
    df_power_clean = df_export[df_export['vermogen'] > 0].copy()
    
    print(f"\n   df_power_clean aangemaakt: {len(df_power_clean)} auto's met vermogen > 0")
    print(f"   Vanaf nu gebruiken we df_power_clean als basis voor alle vervolgstappen")
else:
    print(f"⚠️  BESLISSING: Te weinig auto's met betrouwbaar vermogen")
    print(f"   ({rijen_met_betrouwbaar_vermogen} rijen = {percentage_betrouwbaar:.2f}%)")
    print(f"   We blijven werken met de volledige dataset en imputeren vermogen")
    df_power_clean = df_export.copy()

print("=" * 80)


ANALYSE VERMOGEN - Betrouwbaarheid van vermogen data

Totaal aantal rijen in df_export: 20755
Aantal rijen met NaN in vermogen: 340
Aantal rijen met vermogen == 0: 16868
Percentage met NaN of 0 als vermogen: 82.91%

Aantal rijen met betrouwbaar vermogen (vermogen > 0): 3547
Percentage met betrouwbaar vermogen: 17.09%

--------------------------------------------------------------------------------
⚠️  BESLISSING: Te weinig auto's met betrouwbaar vermogen
   (3547 rijen = 17.09%)
   We blijven werken met de volledige dataset en imputeren vermogen


In [ ]:
# ANALYSE MERK+MODEL: Analyseer unieke combinaties van merk en model in df_power_clean
# Deze analyse helpt bepalen of de dataset voldoende gevarieerde modellen bevat
# voor een recommender die vermogen belangrijk vindt
print("=" * 80)
print("ANALYSE MERK+MODEL - Variatie in auto modellen")
print("=" * 80)

# Bereken basisstatistieken
totaal_rijen = len(df_power_clean)
unieke_merk_model = df_power_clean.groupby(['merk', 'model']).size().reset_index(name='aantal')
aantal_unieke_combinaties = len(unieke_merk_model)

print(f"\nTotaal aantal rijen in df_power_clean: {totaal_rijen}")
print(f"Aantal unieke merk+model combinaties: {aantal_unieke_combinaties}")

# Bereken aantal unieke bouwjaren per merk+model combinatie
merk_model_stats = df_power_clean.groupby(['merk', 'model']).agg({
    'bouwjaar': ['count', 'nunique']
}).reset_index()
merk_model_stats.columns = ['merk', 'model', 'aantal', 'aantal_unieke_bouwjaren']

# Sorteer op aantal (meest voorkomend eerst) en neem top 10
top_10_modellen = merk_model_stats.nlargest(10, 'aantal')

print(f"\nTop 10 meest voorkomende modellen:")
print("-" * 80)
print(f"{'Merk':<20} {'Model':<25} {'Aantal':<10} {'Unieke bouwjaren':<20}")
print("-" * 80)
for idx, row in top_10_modellen.iterrows():
    print(f"{row['merk']:<20} {row['model']:<25} {row['aantal']:<10} {row['aantal_unieke_bouwjaren']:<20}")

# Bereken gemiddeld aantal rijen per merk+model combinatie
gemiddeld_per_combinatie = totaal_rijen / aantal_unieke_combinaties

print("\n" + "-" * 80)
print(f"Gemiddeld aantal rijen per merk+model combinatie: {gemiddeld_per_combinatie:.2f}")

# CONCLUSIE: Bepaal of de dataset voldoende gevarieerd is
print("\n" + "=" * 80)
print("CONCLUSIE:")
print("=" * 80)

# Criteria voor voldoende variatie:
# - Minimaal 100 unieke combinaties voor goede variatie
# - Gemiddeld niet te veel rijen per combinatie (anders te weinig variatie)
# - Top 10 modellen mogen niet te dominant zijn (>30% van totaal)

min_unieke_combinaties = 100
max_gemiddeld_per_combinatie = 50  # Als gemiddeld >50, zijn er te weinig combinaties
max_top10_percentage = 30  # Top 10 mag niet meer dan 30% van totaal zijn

top10_totaal = top_10_modellen['aantal'].sum()
top10_percentage = (top10_totaal / totaal_rijen) * 100

variatie_ok = (aantal_unieke_combinaties >= min_unieke_combinaties) and \
              (gemiddeld_per_combinatie <= max_gemiddeld_per_combinatie) and \
              (top10_percentage <= max_top10_percentage)

if variatie_ok:
    print("✅ De dataset bevat voldoende gevarieerde modellen voor een recommender die vermogen belangrijk vindt.")
    print(f"   - {aantal_unieke_combinaties} unieke merk+model combinaties (aanbevolen: ≥{min_unieke_combinaties})")
    print(f"   - Gemiddeld {gemiddeld_per_combinatie:.2f} rijen per combinatie (aanbevolen: ≤{max_gemiddeld_per_combinatie})")
    print(f"   - Top 10 modellen vormen {top10_percentage:.2f}% van de data (aanbevolen: ≤{max_top10_percentage}%)")
    print("\n   De recommender kan goede variatie bieden tussen verschillende auto modellen.")
else:
    print("⚠️  De dataset heeft mogelijk beperkte variatie in modellen:")
    if aantal_unieke_combinaties < min_unieke_combinaties:
        print(f"   - Te weinig unieke combinaties: {aantal_unieke_combinaties} (aanbevolen: ≥{min_unieke_combinaties})")
    if gemiddeld_per_combinatie > max_gemiddeld_per_combinatie:
        print(f"   - Te veel rijen per combinatie: {gemiddeld_per_combinatie:.2f} (aanbevolen: ≤{max_gemiddeld_per_combinatie})")
    if top10_percentage > max_top10_percentage:
        print(f"   - Top 10 modellen te dominant: {top10_percentage:.2f}% (aanbevolen: ≤{max_top10_percentage}%)")
    print("\n   De recommender kan mogelijk beperkte variatie bieden tussen modellen.")

print("=" * 80)


ANALYSE MERK+MODEL - Variatie in auto modellen

Totaal aantal rijen in df_power_clean: 20755
Aantal unieke merk+model combinaties: 1395

Top 10 meest voorkomende modellen:
--------------------------------------------------------------------------------
Merk                 Model                     Aantal     Unieke bouwjaren    
--------------------------------------------------------------------------------
toyota               prius                     1176       24                  
hyundai              sonata                    1079       12                  
toyota               camry                     1028       24                  
hyundai              elantra                   921        14                  
mercedes-benz        e 350                     542        11                  
honda                fit                       535        24                  
hyundai              santa fe                  533        16                  
hyundai              h1           

In [ ]:
# Stap 4: DEDUPLICATIE - Verwijder exacte duplicaten vóór similarity berekening
# Maak een nieuwe DataFrame met alleen unieke auto's op basis van alle relevante kenmerken
# Dit voorkomt dat exact dezelfde auto's meerdere keren in de similarity matrix voorkomen
# Gebruik df_power_clean als basis (die al gefilterd is op betrouwbaar vermogen als mogelijk)
duplicate_cols = ['merk', 'model', 'brandstof', 'vermogen', 'budget', 'bouwjaar']
aantal_voor = len(df_power_clean)
df_export_unique = df_power_clean.drop_duplicates(subset=duplicate_cols, keep='first').copy()
aantal_na = len(df_export_unique)

print("DEDUPLICATIE - Exacte duplicaten verwijderd:")
print(f"  Aantal rijen voor: {aantal_voor}")
print(f"  Aantal rijen na: {aantal_na}")
print(f"  Verwijderd: {aantal_voor - aantal_na} exacte duplicaten")
print(f"\nVanaf nu werken we met df_export_unique ({len(df_export_unique)} unieke auto's)")

# Maak een kopie voor verdere preprocessing
df_processed = df_export_unique.copy()

# Controleer op missende waarden
print("\nMissende waarden per kolom:")
print(df_processed.isnull().sum())


DEDUPLICATIE - Exacte duplicaten verwijderd:
  Aantal rijen voor: 20755
  Aantal rijen na: 15312
  Verwijderd: 5443 exacte duplicaten

Vanaf nu werken we met df_export_unique (15312 unieke auto's)

Missende waarden per kolom:
merk           0
model          0
bouwjaar       0
type_auto      0
brandstof      0
vermogen     330
budget         0
dtype: int64


In [ ]:
# Stap 5: PREPROCESSING - Label encoding voor categorische variabelen
# Converteer tekstuele categorische variabelen naar numerieke waarden
# Dit is nodig omdat cosine similarity alleen werkt met numerieke waarden
label_encoders = {}

# Encode merk: converteer merk namen naar unieke nummers
if 'merk' in df_processed.columns:
    le_merk = LabelEncoder()
    df_processed['merk_encoded'] = le_merk.fit_transform(df_processed['merk'])
    label_encoders['merk'] = le_merk

# Encode model: converteer model namen naar unieke nummers
if 'model' in df_processed.columns:
    le_model = LabelEncoder()
    df_processed['model_encoded'] = le_model.fit_transform(df_processed['model'])
    label_encoders['model'] = le_model

# Encode brandstof: converteer brandstof types naar unieke nummers
if 'brandstof' in df_processed.columns:
    le_brandstof = LabelEncoder()
    df_processed['brandstof_encoded'] = le_brandstof.fit_transform(df_processed['brandstof'])
    label_encoders['brandstof'] = le_brandstof

print("Label encoding voltooid")


Label encoding voltooid


In [ ]:
# Stap 6: PREPROCESSING - Selecteer en bereid features voor de similarity matrix
# Combineer encoded categorische variabelen met numerieke variabelen
# Deze features worden gebruikt om de gelijkenis tussen auto's te berekenen
# Omdat we df_power_clean gebruiken, hebben alle auto's al betrouwbaar vermogen (> 0)
feature_columns = []

# Voeg encoded categorische features toe (merk, model, brandstof)
if 'merk_encoded' in df_processed.columns:
    feature_columns.append('merk_encoded')
if 'model_encoded' in df_processed.columns:
    feature_columns.append('model_encoded')
if 'brandstof_encoded' in df_processed.columns:
    feature_columns.append('brandstof_encoded')

# Voeg numerieke features toe (vermogen, budget, bouwjaar)
# Vermogen is al gefilterd op > 0 via df_power_clean, dus geen NaN of 0 meer
if 'vermogen' in df_processed.columns:
    feature_columns.append('vermogen')
if 'budget' in df_processed.columns:
    feature_columns.append('budget')
elif 'prijs' in df_processed.columns:
    # Als 'budget' niet bestaat maar 'prijs' wel, gebruik dan 'prijs'
    feature_columns.append('prijs')
if 'bouwjaar' in df_processed.columns:
    feature_columns.append('bouwjaar')

# Maak feature matrix met alleen de geselecteerde kolommen
feature_matrix = df_processed[feature_columns].copy()

# Controle: als er nog steeds missende waarden zijn (onwaarschijnlijk na df_power_clean),
# vul ze dan met mediaan als backup
if 'vermogen' in feature_matrix.columns:
    if feature_matrix['vermogen'].isna().sum() > 0:
        feature_matrix['vermogen'] = feature_matrix['vermogen'].fillna(feature_matrix['vermogen'].median())
        print("⚠️  Waarschuwing: Er waren nog steeds missende waarden in vermogen (backup imputatie toegepast)")

print(f"Feature kolommen: {feature_columns}")
print(f"Feature matrix shape: {feature_matrix.shape}")
print(f"Missende waarden na preprocessing: {feature_matrix.isnull().sum().sum()}")


⚠️  Waarschuwing: Er waren nog steeds missende waarden in vermogen (backup imputatie toegepast)
Feature kolommen: ['merk_encoded', 'model_encoded', 'brandstof_encoded', 'vermogen', 'budget', 'bouwjaar']
Feature matrix shape: (15312, 6)
Missende waarden na preprocessing: 0


In [ ]:
# Stap 7: SCALING - Normaliseer de features voor cosine similarity
# Normalisatie is cruciaal omdat features verschillende schalen hebben
# (bijv. bouwjaar: 2000-2020, budget: duizenden euros, vermogen: variabel)
# StandardScaler transformeert alle features naar gemiddelde=0 en standaarddeviatie=1
scaler = StandardScaler()
feature_matrix_scaled = scaler.fit_transform(feature_matrix)
feature_matrix_scaled = pd.DataFrame(
    feature_matrix_scaled, 
    columns=feature_columns,
    index=df_processed.index
)

print("Features genormaliseerd (gemiddelde=0, standaarddeviatie=1)")
print(feature_matrix_scaled.head())


Features genormaliseerd (gemiddelde=0, standaarddeviatie=1)
   merk_encoded  model_encoded  brandstof_encoded  vermogen    budget  \
0      0.056194       1.020961          -0.507749 -0.124272 -0.459383   
1     -1.277563      -0.453137           0.864410 -0.124272 -0.331904   
2     -0.555111      -0.367990           0.864410 -0.124272 -0.647562   
3     -0.832977      -0.437172          -0.507749 -0.124272 -0.835703   
4     -0.555111      -0.367990           0.864410 -0.124272 -0.521400   

   bouwjaar  
0 -0.285106  
1 -0.087104  
2 -1.077114  
3 -0.087104  
4  0.506902  


In [ ]:
# Stap 8: SIMILARITY - Bereken de cosine similarity matrix
# Cosine similarity meet de hoek tussen twee feature vectoren
# Waarde tussen -1 en 1: 1 = identiek, 0 = orthogonaal, -1 = tegengesteld
# Voor normalisatie geeft dit een waarde tussen 0 en 1 (meestal)
similarity_matrix = cosine_similarity(feature_matrix_scaled)
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=df_processed.index,
    columns=df_processed.index
)

print("Similarity matrix berekend")
print(f"Shape: {similarity_df.shape}")
print("\nVoorbeeld similarity scores (eerste 5x5):")
print(similarity_df.iloc[:5, :5])


Similarity matrix berekend
Shape: (15312, 15312)

Voorbeeld similarity scores (eerste 5x5):
          0         1         2         3         4
0  1.000000 -0.373028 -0.106578  0.108971 -0.439831
1 -0.373028  1.000000  0.707711  0.499250  0.813932
2 -0.106578  0.707711  1.000000  0.366325  0.453888
3  0.108971  0.499250  0.366325  1.000000  0.328834
4 -0.439831  0.813932  0.453888  0.328834  1.000000


In [12]:
# Deze cel is verwijderd - code is verplaatst naar eerdere cellen



In [13]:
# Stap 9: AANBEVELINGEN - Functie voor het genereren van aanbevelingen
def get_recommendations(item_index=None, merk=None, model=None, bouwjaar=None, 
                       similarity_df=None, df_original=None, n_recommendations=5):
    """
    Geef aanbevelingen op basis van een item index OF (merk, model, bouwjaar).
    
    Parameters:
    - item_index: index van het item waarvoor we aanbevelingen willen (optioneel)
    - merk: merk naam (optioneel, gebruikt samen met model en bouwjaar)
    - model: model naam (optioneel, gebruikt samen met merk en bouwjaar)
    - bouwjaar: bouwjaar (optioneel, gebruikt samen met merk en model)
    - similarity_df: dataframe met similarity scores
    - df_original: originele dataframe met alle informatie
    - n_recommendations: aantal aanbevelingen (default: 5)
    
    Returns:
    - DataFrame met aanbevelingen gesorteerd op similarity score
    """
    # Bepaal item_index: gebruik directe index OF zoek op basis van merk/model/bouwjaar
    if item_index is None:
        if merk is None or model is None or bouwjaar is None:
            raise ValueError("Geef óf item_index, óf (merk, model, bouwjaar) op")
        
        # Zoek het item op basis van merk, model en bouwjaar
        filtered = df_original[(df_original['merk'] == merk) & 
                               (df_original['model'] == model) & 
                               (df_original['bouwjaar'] == bouwjaar)]
        
        if len(filtered) == 0:
            raise ValueError(f"Geen auto gevonden met merk={merk}, model={model}, bouwjaar={bouwjaar}")
        
        item_index = filtered.index[0]
    
    # Haal similarity scores op voor dit item en sorteer van hoog naar laag
    similarities = similarity_df.loc[item_index].sort_values(ascending=False)
    
    # FILTERING - Verwijder het gekozen item zelf (similarity = 1.0)
    similarities = similarities[similarities.index != item_index]
    
    # FILTERING - Sluit alle auto's uit met hetzelfde merk, model en bouwjaar
    # Dit voorkomt dat auto's van hetzelfde type worden aanbevolen, zelfs als budget/vermogen anders is
    item_data = df_original.loc[item_index]
    same_model_mask = (df_original['merk'] == item_data['merk']) & \
                      (df_original['model'] == item_data['model']) & \
                      (df_original['bouwjaar'] == item_data['bouwjaar'])
    
    # Filter de similarity scores: verwijder alle indices die voldoen aan same_model_mask
    indices_to_exclude = df_original[same_model_mask].index
    similarities = similarities[~similarities.index.isin(indices_to_exclude)]
    
    # Haal de originele data op voor alle overgebleven kandidaten
    all_candidates = df_original.loc[similarities.index].copy()
    all_candidates['similarity_score'] = similarities.values
    
    # Neem de top n aanbevelingen (gesorteerd op similarity score)
    recommendations = all_candidates.head(n_recommendations)
    
    return recommendations

print("Recommendation functie gedefinieerd")


Recommendation functie gedefinieerd


In [14]:
# TESTSECTIE: Test de recommender en controleer dat geen aanbevelingen hetzelfde merk+model+bouwjaar hebben
# Alle aanbevelingen bevatten alleen auto's met betrouwbaar vermogen (vermogen > 0)
print("=" * 80)
print("TESTSECTIE - Content-based Recommender")
print("=" * 80)

# Kies een voorbeeld index
example_index = 0

# Haal aanbevelingen op
recommendations = get_recommendations(
    item_index=example_index,
    similarity_df=similarity_df,
    df_original=df_export_unique,  # Gebruik df_export_unique (zonder duplicaten, met betrouwbaar vermogen)
    n_recommendations=5
)

# Definieer kolommen voor weergave (inclusief type_auto)
display_cols_orig = ['merk', 'model', 'type_auto', 'vermogen', 'brandstof', 'budget', 'bouwjaar']
display_cols_orig = [col for col in display_cols_orig if col in df_export_unique.columns]
display_cols = ['merk', 'model', 'type_auto', 'vermogen', 'brandstof', 'budget', 'bouwjaar', 'similarity_score']
display_cols = [col for col in display_cols if col in recommendations.columns]

# Print het originele item
print(f"\nOrigineel item (index {example_index}):")
original_item = df_export_unique.loc[example_index]
print(original_item[display_cols_orig])

# Print de top 5 aanbevelingen
print(f"\nTop 5 aanbevelingen (allemaal met vermogen > 0):")
print(recommendations[display_cols])

# VALIDATIE 1: Controleer dat geen van de aanbevelingen hetzelfde merk+model+bouwjaar heeft
print("\n" + "-" * 80)
print("VALIDATIE 1: Controle op uitsluiting van zelfde merk+model+bouwjaar")
print("-" * 80)

original_merk = original_item['merk']
original_model = original_item['model']
original_bouwjaar = original_item['bouwjaar']

# Check elke aanbeveling
validation_passed = True
for idx, rec in recommendations.iterrows():
    same_combo = (rec['merk'] == original_merk) and \
                 (rec['model'] == original_model) and \
                 (rec['bouwjaar'] == original_bouwjaar)
    
    if same_combo:
        print(f"❌ FOUT: Aanbeveling {idx} heeft hetzelfde merk+model+bouwjaar als origineel!")
        print(f"   Merk: {rec['merk']}, Model: {rec['model']}, Bouwjaar: {rec['bouwjaar']}")
        validation_passed = False

if validation_passed:
    print("✅ VALIDATIE GESLAAGD: Geen enkele aanbeveling heeft hetzelfde merk+model+bouwjaar als het originele item")
    print(f"   Origineel: {original_merk} {original_model} ({original_bouwjaar})")
    print(f"   Alle {len(recommendations)} aanbevelingen hebben verschillende combinaties")

# VALIDATIE 2: Controleer dat alle aanbevelingen betrouwbaar vermogen hebben
print("\n" + "-" * 80)
print("VALIDATIE 2: Controle dat alle aanbevelingen betrouwbaar vermogen hebben (vermogen > 0)")
print("-" * 80)

validation_vermogen_passed = True
for idx, rec in recommendations.iterrows():
    if rec['vermogen'] <= 0 or pd.isna(rec['vermogen']):
        print(f"❌ FOUT: Aanbeveling {idx} heeft geen betrouwbaar vermogen!")
        print(f"   Vermogen: {rec['vermogen']}")
        validation_vermogen_passed = False

if validation_vermogen_passed:
    print("✅ VALIDATIE GESLAAGD: Alle aanbevelingen hebben betrouwbaar vermogen (vermogen > 0)")
    min_vermogen = recommendations['vermogen'].min()
    max_vermogen = recommendations['vermogen'].max()
    print(f"   Vermogen range in aanbevelingen: {min_vermogen:.2f} - {max_vermogen:.2f}")

print("\n" + "=" * 80)
print("Testsectie voltooid")
print("=" * 80)


TESTSECTIE - Content-based Recommender

Origineel item (index 0):
merk           lexus
model         rx 450
type_auto       jeep
vermogen         0.0
brandstof     hybrid
budget       13328.0
bouwjaar        2010
Name: 0, dtype: object

Top 5 aanbevelingen (allemaal met vermogen > 0):
                merk   model type_auto  vermogen brandstof  budget  bouwjaar  \
13746          lexus  rx 450      jeep      0.00    hybrid  7997.0      2008   
2998           lexus  rx 400      jeep      0.00    hybrid  8154.0      2008   
374            lexus  rx 400      jeep      0.00    hybrid  7997.0      2008   
6406           lexus  rx 300      jeep      0.00    hybrid  8154.0      2008   
1524   mercedes-benz   viano   minivan      0.75    diesel   120.0      2007   

       similarity_score  
13746          0.960440  
2998           0.959646  
374            0.959372  
6406           0.958827  
1524           0.951036  

----------------------------------------------------------------------------

In [15]:
# SAMENVATTING: Overzicht van gebruikte data voor de recommender
print("=" * 80)
print("SAMENVATTING - Data gebruikt in de recommender")
print("=" * 80)

# Bereken statistieken
totaal_auto_s = len(df_export)
auto_s_met_vermogen = len(df_power_clean)
percentage_gebruikt = (auto_s_met_vermogen / totaal_auto_s) * 100

print(f"\nTotaal aantal auto's: {totaal_auto_s}")
print(f"Aantal auto's met vermogen > 0: {auto_s_met_vermogen}")
print(f"Percentage gebruikt in de recommender: {percentage_gebruikt:.2f}%")
print(f"\nAuto's zonder vermogen worden NIET gebruikt in de similarity-berekening.")

if len(df_power_clean) < len(df_export):
    auto_s_uitgesloten = totaal_auto_s - auto_s_met_vermogen
    print(f"\n⚠️  Let op: {auto_s_uitgesloten} auto's zijn uitgesloten omdat ze geen betrouwbaar vermogen hebben")
    print(f"   (NaN of vermogen == 0)")

print("\n" + "=" * 80)


SAMENVATTING - Data gebruikt in de recommender

Totaal aantal auto's: 20755
Aantal auto's met vermogen > 0: 20755
Percentage gebruikt in de recommender: 100.00%

Auto's zonder vermogen worden NIET gebruikt in de similarity-berekening.



In [16]:
# Optionele helper functie (voor achterwaartse compatibiliteit)
# Deze functie is nu minder nodig omdat get_recommendations direct merk/model/bouwjaar ondersteunt
def get_recommendations_by_name(merk_value=None, model_value=None, similarity_df=None, 
                                df_original=None, n_recommendations=5):
    """
    Helper functie voor aanbevelingen op basis van merk en/of model naam.
    Let op: deze functie gebruikt alleen het eerste gevonden item.
    Gebruik liever get_recommendations met merk, model en bouwjaar voor betere resultaten.
    """
    # Filter dataframe op basis van merk/model
    filtered_df = df_original.copy()
    
    if merk_value:
        filtered_df = filtered_df[filtered_df['merk'] == merk_value]
    if model_value:
        filtered_df = filtered_df[filtered_df['model'] == model_value]
    
    if len(filtered_df) == 0:
        print("Geen items gevonden met deze criteria")
        return None
    
    # Neem het eerste item uit de gefilterde lijst
    item_index = filtered_df.index[0]
    
    # Gebruik de bestaande recommendation functie
    return get_recommendations(item_index, similarity_df, df_original, n_recommendations)

print("Helper functie voor aanbevelingen op basis van naam gedefinieerd")


Helper functie voor aanbevelingen op basis van naam gedefinieerd


In [17]:
# Optionele extra voorbeelden (uncomment om te gebruiken):
# 
# # Voorbeeld 1: Gebruik get_recommendations_by_name (oude methode)
# recommendations_by_name = get_recommendations_by_name(
#     merk_value='toyota',
#     similarity_df=similarity_df,
#     df_original=df_processed,
#     n_recommendations=5
# )
# if recommendations_by_name is not None:
#     display_cols = ['merk', 'model', 'vermogen', 'brandstof', 'budget', 'bouwjaar', 'similarity_score']
#     display_cols = [col for col in display_cols if col in recommendations_by_name.columns]
#     print(recommendations_by_name[display_cols])
#
# # Voorbeeld 2: Gebruik get_recommendations met merk/model/bouwjaar (nieuwe methode - aanbevolen)
# recommendations_new = get_recommendations(
#     merk='honda',
#     model='civic',
#     bouwjaar=2016,
#     similarity_df=similarity_df,
#     df_original=df_processed,
#     n_recommendations=5
# )
# print(recommendations_new[display_cols])
